In [49]:
import pandas as pd

# Ruta a tu archivo Parquet
hydro = 'core_ferc1__yearly_hydroelectric_plants_sched406.parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df = pd.read_parquet(hydro)

    # Ahora puedes trabajar con el DataFrame
    print("Archivo Parquet cargado exitosamente.")
    print("Primeras 5 filas del DataFrame:")
    print(df.head())
    print("\nInformación del DataFrame:")
    df.info()

except FileNotFoundError:
    print(f"Error: El archivo '{hydro}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

Archivo Parquet cargado exitosamente.
Primeras 5 filas del DataFrame:
                record_id  utility_id_ferc1  report_year    plant_name_ferc1  project_num    plant_type construction_type  construction_year  installation_year  capacity_mw  peak_demand_mw  plant_hours_connected_while_generating  net_capacity_favorable_conditions_mw  net_capacity_adverse_conditions_mw  avg_num_employees  net_generation_mwh  capex_land  capex_structures  capex_facilities  capex_equipment  capex_roads  asset_retirement_cost  capex_total  capex_per_mw  opex_operations  opex_water_for_power  opex_hydraulic  opex_electric  opex_generation_misc  opex_rents  opex_engineering  opex_structures  opex_dams  opex_plant  opex_misc_plant  opex_total  opex_per_mwh
0  f1_hydro_1994_12_2_0_1               294         1994           weiss dam       2146.0       storage      conventional             1961.0             1962.0    87.800003            78.0                                  4377.0                           

In [58]:
import pandas as pd

# Ruta a tu archivo Parquet
ruta_archivo_parquet = 'core_ferc1__yearly_steam_plants_sched402 (1).parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df2 = pd.read_parquet(ruta_archivo_parquet)

    # Ahora puedes trabajar con el DataFrame
    print("Archivo Parquet cargado exitosamente.")
    print("Primeras 5 filas del DataFrame:")
    print(df.head())
    print("\nInformación del DataFrame:")
    df2.info()

except FileNotFoundError:
    print(f"Error: El archivo '{ruta_archivo_parquet}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

Archivo Parquet cargado exitosamente.
Primeras 5 filas del DataFrame:
                record_id  utility_id_ferc1  report_year    plant_name_ferc1  project_num    plant_type construction_type  construction_year  installation_year  capacity_mw  peak_demand_mw  plant_hours_connected_while_generating  net_capacity_favorable_conditions_mw  net_capacity_adverse_conditions_mw  avg_num_employees  net_generation_mwh  capex_land  capex_structures  capex_facilities  capex_equipment  capex_roads  asset_retirement_cost  capex_total  capex_per_mw  opex_operations  opex_water_for_power  opex_hydraulic  opex_electric  opex_generation_misc  opex_rents  opex_engineering  opex_structures  opex_dams  opex_plant  opex_misc_plant  opex_total  opex_per_mwh
0  f1_hydro_1994_12_2_0_1               294         1994           weiss dam       2146.0       storage      conventional             1961.0             1962.0    87.800003            78.0                                  4377.0                           

In [78]:
df 

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,project_num,plant_type,construction_type,construction_year,installation_year,capacity_mw,peak_demand_mw,plant_hours_connected_while_generating,net_capacity_favorable_conditions_mw,net_capacity_adverse_conditions_mw,avg_num_employees,net_generation_mwh,capex_land,capex_structures,capex_facilities,capex_equipment,capex_roads,asset_retirement_cost,capex_total,capex_per_mw,opex_operations,opex_water_for_power,opex_hydraulic,opex_electric,opex_generation_misc,opex_rents,opex_engineering,opex_structures,opex_dams,opex_plant,opex_misc_plant,opex_total,opex_per_mwh
0,f1_hydro_1994_12_2_0_1,294,1994,weiss dam,2146.0,storage,conventional,1961.0,1962.0,87.800003,78.0,4377.0,90.0,51.0,11.0,264862.0,12311296.0,4301538.0,15069191.0,10133498.0,NaN,NaN,41815524.0,NaN,153153.0,38410.0,109148.0,6495.0,357552.0,5.0,132735.0,58916.0,90591.0,82675.0,20596.0,1050276.0,4.0
1,f1_hydro_1994_12_2_0_2,294,1994,h. neely henry dam,2146.0,storage,conventional,1966.0,1966.0,72.900002,69.0,4816.0,77.0,9.0,14.0,246845.0,9923654.0,1409693.0,14360910.0,7981002.0,NaN,NaN,33675260.0,NaN,187681.0,31413.0,114610.0,8637.0,384100.0,6.0,124074.0,103097.0,81941.0,96102.0,43331.0,1174992.0,4.8
2,f1_hydro_1994_12_2_0_3,294,1994,logan martin dam,2146.0,storage,conventional,1964.0,1964.0,128.300003,132.0,5050.0,148.0,107.0,12.0,462313.0,20082140.0,2539894.0,47827104.0,10283579.0,NaN,NaN,80732712.0,NaN,177678.0,55712.0,118886.0,27523.0,344836.0,8408.0,125759.0,108923.0,405834.0,114688.0,18842.0,1507089.0,3.3
3,f1_hydro_1994_12_2_0_4,294,1994,lay dam,2146.0,run_of_river,conventional,1914.0,1968.0,177.000000,185.0,5402.0,194.0,144.0,11.0,719686.0,7732752.0,1639252.0,17370788.0,11926333.0,NaN,NaN,38669124.0,NaN,195818.0,75913.0,112752.0,22128.0,360393.0,6.0,102792.0,57437.0,21990.0,165671.0,35750.0,1150650.0,1.6
4,f1_hydro_1994_12_2_0_5,294,1994,mitchell dam,82.0,run_of_river,conventional,1923.0,1985.0,170.000000,169.0,5446.0,176.0,122.0,14.0,598137.0,511738.0,23245022.0,110799656.0,87100656.0,2215506.0,NaN,223872576.0,NaN,194087.0,72913.0,82603.0,17199.0,394073.0,8019.0,140501.0,55660.0,37687.0,122042.0,26232.0,1151016.0,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7416,hydroelectric_generating_plant_statistics_larg...,295,2024,tallulah falls,2354.0,run_of_river,conventional,1913.0,1920.0,67.000000,58.0,3307.0,73.0,73.0,NaN,71817.0,409265.0,3963761.0,13353706.0,21487988.0,1534469.0,1231223.0,41980412.0,626600.0,196547.0,NaN,72.0,59944.0,41132.0,NaN,-14.0,292416.0,1297.0,1233011.0,10483.0,1834888.0,25.5
7417,hydroelectric_generating_plant_statistics_larg...,295,2024,tugalo,2354.0,run_of_river,conventional,1923.0,1924.0,45.000000,34.0,6423.0,52.0,52.0,NaN,96921.0,196578.0,3983776.0,25567958.0,74841272.0,561750.0,1250920.0,106402256.0,2364500.0,156251.0,NaN,59.0,3325.0,36128.0,NaN,-11.0,161294.0,31154.0,193724.0,NaN,581924.0,6.0
7418,hydroelectric_generating_plant_statistics_larg...,295,2024,yonah,2354.0,run_of_river,conventional,1925.0,1925.0,22.500000,24.0,5997.0,29.0,29.0,NaN,42125.0,41465.0,1246608.0,6499642.0,6526153.0,43277.0,920612.0,15277757.0,679000.0,59504.0,NaN,22.0,17071.0,14257.0,NaN,-4.0,61351.0,5776.0,187004.0,NaN,344981.0,8.2
7419,hydroelectric_generating_plant_statistics_larg...,295,2024,wallace dam,2413.0,storage,semioutdoor,1979.0,1980.0,112.500000,226.0,4714.0,115.0,115.0,NaN,195051.0,8734309.0,13269318.0,51781720.0,8991692.0,638184.0,25903.0,83441120.0,741700.0,401967.0,NaN,134.0,525137.0,93805.0,21736.0,-26.0,406461.0,126656.0,593274.0,36484.0,2205628.0,11.3


In [76]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos (MEJORADA)
def prepare_data(df):
    """
    Prepara los datos asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime (manejo de diferentes formatos)
    if not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            # Primero intentar convertir a entero y luego a datetime
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            # Si falla, intentar convertir directamente
            df['report_year'] = pd.to_datetime(df['report_year'])
    
    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total']
        df['capex_total'] = df[capex_cols].sum(axis=1)
    
    if 'opex_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and col != 'opex_total']
        df['opex_total'] = df[opex_cols].sum(axis=1)
    
    # Calcular ratios con protección contra división por cero
    df['capex_per_mw'] = df.apply(
        lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
        axis=1
    )
    
    df['opex_per_mwh'] = df.apply(
        lambda x: x['opex_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
        axis=1
    )
    
    # Asegurar que plant_type sea string y manejar valores nulos
    df['plant_type'] = df['plant_type'].astype(str)
    df['plant_type'] = df['plant_type'].replace('nan', 'No especificado')
    
    # Asegurar que plant_name_ferc1 sea string y manejar valores nulos
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str)
        df['plant_name_ferc1'] = df['plant_name_ferc1'].replace('nan', 'No especificado')
        # Crear columna combinada para mostrar en el selector
        df['plant_name_with_type'] = df['plant_name_ferc1'] + ' (' + df['plant_type'] + ')'
    
    return df

## 2. Evolución de CAPEX, OPEX y Capacidad (MEJORADA)
def plot_cost_evolution(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data(df)
    
    # Convertir años seleccionados a datetime para comparación
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Agrupar por año
    yearly_data = filtered_df.groupby('report_year').agg({
        'capex_total': 'sum',
        'opex_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }).reset_index()
    
    # Calcular ratios
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    # Crear figura
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # CAPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # Capacidad (Línea)
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[2], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),  # Leyenda centrada
        margin=dict(t=80, b=80),  # Margen superior aumentado
        xaxis=dict(
            tickformat='%Y',  # Mostrar solo el año
            type='date'  # Asegurar que se trate como fecha
        )
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX) y operación (OPEX), 
        junto con la capacidad instalada total. Los valores de costos se muestran en USD (eje izquierdo) 
        mientras que la capacidad se muestra en MW (eje derecho).</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos para evitar 
        interpretaciones erróneas con números negativos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada (MEJORADA)
def plot_cost_composition(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data(df)
    
    # Convertir años seleccionados a datetime para comparación
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Componentes CAPEX (excluyendo totales)
    capex_cols = [col for col in df_clean.columns 
                 if col.startswith('capex_') 
                 and col not in ['capex_total', 'capex_per_mw']
                 and df_clean[col].sum() > 0]  # Solo columnas con datos
    
    # Componentes OPEX (excluyendo totales)
    opex_cols = [col for col in df_clean.columns 
                if col.startswith('opex_') 
                and col not in ['opex_total', 'opex_per_mwh']
                and df_clean[col].sum() > 0]  # Solo columnas con datos
    
    # Crear figura con subplots
    fig = make_subplots(rows=2, cols=1, 
                       subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                       vertical_spacing=0.15)
    
    # ---- Gráfico CAPEX ----
    capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
    capex_pct = capex_data.set_index('plant_type').div(capex_data.set_index('plant_type').sum(axis=1), axis=0) * 100
    
    for i, col in enumerate(capex_cols):
        # Texto para mostrar en las barras (valor USD + porcentaje)
        text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                        zip(capex_data[col], capex_pct[col])]
        
        fig.add_trace(
            go.Bar(
                x=capex_data['plant_type'],
                y=capex_data[col],
                name=col.replace('capex_', '').replace('_', ' ').title(),
                marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                text=text_template,
                textposition='auto',
                hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                customdata=capex_pct[col],
                textfont=dict(size=10),
                showlegend=False  # Ocultar leyenda ya que la info está en las etiquetas
            ),
            row=1, col=1
        )
    
    # ---- Gráfico OPEX ----
    opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
    opex_pct = opex_data.set_index('plant_type').div(opex_data.set_index('plant_type').sum(axis=1), axis=0) * 100
    
    for i, col in enumerate(opex_cols):
        # Texto para mostrar en las barras (valor USD + porcentaje)
        text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                        zip(opex_data[col], opex_pct[col])]
        
        fig.add_trace(
            go.Bar(
                x=opex_data['plant_type'],
                y=opex_data[col],
                name=col.replace('opex_', '').replace('_', ' ').title(),
                marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                text=text_template,
                textposition='auto',
                hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                customdata=opex_pct[col],
                textfont=dict(size=10),
                showlegend=False  # Ocultar leyenda ya que la info está en las etiquetas
            ),
            row=2, col=1
        )
    
    # Actualizar diseño
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80)  # Eliminamos la leyenda del layout
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        
        <p><b>Características:</b></p>
        <ul>
            <li><b>Gráficos separados:</b> CAPEX y OPEX mostrados en gráficos independientes para mayor claridad</li>
            <li><b>Valores absolutos:</b> Eje Y muestra valores en USD</li>
            <li><b>Porcentajes:</b> Cada barra muestra el valor en USD y el porcentaje que representa</li>
            <li><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen con Explicación (MEJORADA)
def display_summary_table(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data(df)
    
    # Convertir años seleccionados a datetime para comparación
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Calcular métricas resumen
    summary = filtered_df.groupby('plant_type').agg({
        'capex_total': ['sum', 'mean'],
        'opex_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    })
    
    # Calcular ratios con protección contra división por cero
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    # Formatear tabla
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    summary = summary[[
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_total sum',
        'opex_total mean',
        'opex_per_mwh'
    ]]
    
    # Renombrar columnas
    summary.columns = [
        'Capacidad Total (MW)',
        'Generación Total (MWh)',
        'CAPEX Total (USD)',
        'CAPEX Promedio (USD)',
        'CAPEX por MW (USD/MW)',
        'OPEX Total (USD)',
        'OPEX Promedio (USD)',
        'OPEX por MWh (USD/MWh)'
    ]
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados. Los valores incluyen:</p>
        <ul>
            <li><b>CAPEX Total/Promedio:</b> Inversión total en capital y promedio por planta</li>
            <li><b>CAPEX por MW:</b> Costo de capital por unidad de capacidad instalada</li>
            <li><b>OPEX Total/Promedio:</b> Costos operativos totales y promedio por planta</li>
            <li><b>OPEX por MWh:</b> Costo operativo por unidad de energía generada</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    
    # Mostrar tabla con formato mejorado
    styled_table = summary.style.format({
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }).background_gradient(cmap='Blues', subset=['CAPEX Total (USD)', 'OPEX Total (USD)'])
    
    # Resaltar métricas clave
    styled_table = styled_table.set_properties(
        subset=['CAPEX por MW (USD/MW)', 'OPEX por MWh (USD/MWh)'],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos"))

## Interfaz de Control Mejorada (MEJORADA)
def create_advanced_dashboard(df):
    # Preparar datos
    df_clean = prepare_data(df)
    
    # Obtener opciones para los widgets
    available_years = sorted(df_clean['report_year'].dt.year.unique())
    
    # Manejar tipos de planta (asegurar que sean strings)
    plant_types = df_clean['plant_type'].unique()
    available_plant_types = sorted([str(pt) for pt in plant_types])
    
    # Manejar nombres de plantas (si existe la columna)
    if 'plant_name_with_type' in df_clean.columns:
        plant_names = df_clean['plant_name_with_type'].unique()
        available_plant_names = sorted([str(pn) for pn in plant_names if str(pn) != 'nan'])
    elif 'plant_name_ferc1' in df_clean.columns:
        plant_names = df_clean['plant_name_ferc1'].unique()
        available_plant_names = sorted([str(pn) for pn in plant_names if str(pn) != 'nan'])
    else:
        available_plant_names = ['No disponible']
    
    # Widgets con opción "Todos"
    year_selector = widgets.SelectMultiple(
        options=['Todos'] + available_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + available_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + available_plant_names,
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    # Función para manejar la selección "Todos"
    def get_selected_options(selected, all_options):
        if 'Todos' in selected or len(selected) == 0:
            return all_options
        return selected
    
    # Función para extraer solo el nombre de la planta (sin el tipo entre paréntesis)
    def extract_plant_name(plant_with_type):
        if '(' in plant_with_type:
            return plant_with_type.split('(')[0].strip()
        return plant_with_type
    
    # Función de actualización
    def update_analysis(analysis_type, selected_years, selected_plant_types, selected_plants):
        # Manejar selección "Todos"
        years_to_use = get_selected_options(selected_years, available_years)
        plant_types_to_use = get_selected_options(selected_plant_types, available_plant_types)
        plants_to_use = get_selected_options(selected_plants, available_plant_names)
        
        # Convertir años a enteros para el filtrado (la función prepare_data los convertirá a datetime)
        years_to_use = [int(year) if year != 'Todos' and str(year).isdigit() else year for year in years_to_use]
        
        # Extraer solo el nombre de la planta (sin el tipo entre paréntesis)
        plants_to_use_names = [extract_plant_name(p) for p in plants_to_use]
        
        if analysis_type == 'evolution':
            plot_cost_evolution(df_clean, years_to_use, plant_types_to_use, plants_to_use_names)
        elif analysis_type == 'composition':
            plot_cost_composition(df_clean, years_to_use, plant_types_to_use, plants_to_use_names)
        elif analysis_type == 'summary':
            display_summary_table(df_clean, years_to_use, plant_types_to_use, plants_to_use_names)
    
    # Interfaz interactiva
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector, plant_name_selector]),
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type': analysis_type,
            'selected_years': year_selector,
            'selected_plant_types': plant_type_selector,
            'selected_plants': plant_name_selector
        }
    )
    
    display(ui, out)

# Ejecutar el dashboard
create_advanced_dashboard(df)

Output()

In [53]:
print(df2.columns)

Index(['record_id', 'utility_id_ferc1', 'report_year', 'plant_name_ferc1', 'plant_type', 'construction_type', 'construction_year', 'installation_year', 'capacity_mw', 'peak_demand_mw', 'plant_hours_connected_while_generating', 'plant_capability_mw', 'water_limited_capacity_mw', 'not_water_limited_capacity_mw', 'avg_num_employees', 'net_generation_mwh', 'capex_land', 'capex_structures', 'capex_equipment', 'capex_total', 'capex_per_mw', 'opex_operations', 'opex_fuel', 'opex_coolants', 'opex_steam', 'opex_steam_other', 'opex_transfer', 'opex_electric', 'opex_misc_power', 'opex_rents', 'opex_allowances', 'opex_engineering', 'opex_structures', 'opex_boiler', 'opex_plants', 'opex_misc_steam', 'opex_production_total', 'opex_per_mwh', 'asset_retirement_cost'], dtype='object')


In [93]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos para df2
def prepare_data_df2(df):
    """
    Prepara los datos de df2 asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime
    if not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            df['report_year'] = pd.to_datetime(df['report_year'])
    
    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        if col in df.columns:
            df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total']
        df['capex_total'] = df[capex_cols].sum(axis=1)
    
    if 'opex_production_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and 
                    col not in ['opex_production_total', 'opex_per_mwh', 'opex_transfer']]
        df['opex_production_total'] = df[opex_cols].sum(axis=1)
    
    # Calcular ratios
    df['capex_per_mw'] = df.apply(
        lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
        axis=1
    )
    
    df['opex_per_mwh'] = df.apply(
        lambda x: x['opex_production_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
        axis=1
    )
    
    # Asegurar que plant_type sea string
    df['plant_type'] = df['plant_type'].astype(str).replace('nan', 'No especificado')
    
    # Asegurar que plant_name_ferc1 sea string y agregar tipo de planta entre paréntesis
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str).replace('nan', 'No especificado')
        # Crear columna combinada para mostrar en el selector
        df['plant_name_with_type'] = df['plant_name_ferc1'] + ' (' + df['plant_type'] + ')'
    
    return df

## 2. Evolución de Costos y Capacidad para df2
def plot_cost_evolution_df2(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data_df2(df)
    
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Agrupar por año
    agg_dict = {
        'capex_total': 'sum',
        'opex_production_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }
    
    # Solo agregar opex_fuel si existe en el DataFrame
    if 'opex_fuel' in df_clean.columns:
        agg_dict['opex_fuel'] = 'sum'
    
    yearly_data = filtered_df.groupby('report_year').agg(agg_dict).reset_index()
    
    # Calcular ratios
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_production_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    # Crear figura
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # CAPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_production_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX Fuel (Barras) si existe
    if 'opex_fuel' in yearly_data.columns:
        fig.add_trace(
            go.Bar(
                x=yearly_data['report_year'],
                y=yearly_data['opex_fuel'],
                name='Costos Combustible (USD)',
                marker_color=COLOR_PALETTE[2],
                opacity=0.7,
                hovertemplate='<b>Combustible</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
            ),
            secondary_y=False
        )
    
    # Capacidad (Línea)
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[3], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),
        margin=dict(t=80, b=80),  # Aumentar margen superior para evitar superposición
        xaxis=dict(tickformat='%Y', type='date')
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX), 
        operación (OPEX) y capacidad instalada.</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada para df2
def plot_cost_composition_df2(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data_df2(df)
    
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Componentes CAPEX
    capex_cols = [col for col in df_clean.columns 
                 if col.startswith('capex_') 
                 and col not in ['capex_total', 'capex_per_mw']
                 and col in df_clean.columns
                 and df_clean[col].sum() > 0]
    
    # Componentes OPEX
    opex_cols = [col for col in df_clean.columns 
                if col.startswith('opex_') 
                and col not in ['opex_production_total', 'opex_per_mwh', 'opex_transfer']
                and col in df_clean.columns
                and df_clean[col].sum() > 0]
    
    # Crear figura con subplots
    fig = make_subplots(rows=2, cols=1, 
                       subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                       vertical_spacing=0.15)
    
    # ---- Gráfico CAPEX ----
    if capex_cols:
        capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
        capex_pct = capex_data.set_index('plant_type').div(capex_data.set_index('plant_type').sum(axis=1), axis=0) * 100
        
        for i, col in enumerate(capex_cols):
            text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                            zip(capex_data[col], capex_pct[col])]
            
            fig.add_trace(
                go.Bar(
                    x=capex_data['plant_type'],
                    y=capex_data[col],
                    name=col.replace('capex_', '').replace('_', ' ').title(),
                    marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                    text=text_template,
                    textposition='auto',
                    hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                    customdata=capex_pct[col],
                    textfont=dict(size=10),
                    showlegend=False  # Ocultar leyenda ya que la info está en las etiquetas
                ),
                row=1, col=1
            )
    
    # ---- Gráfico OPEX ----
    if opex_cols:
        opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
        opex_pct = opex_data.set_index('plant_type').div(opex_data.set_index('plant_type').sum(axis=1), axis=0) * 100
        
        for i, col in enumerate(opex_cols):
            text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                            zip(opex_data[col], opex_pct[col])]
            
            fig.add_trace(
                go.Bar(
                    x=opex_data['plant_type'],
                    y=opex_data[col],
                    name=col.replace('opex_', '').replace('_', ' ').title(),
                    marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                    text=text_template,
                    textposition='auto',
                    hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                    customdata=opex_pct[col],
                    textfont=dict(size=10),
                    showlegend=False  # Ocultar leyenda ya que la info está en las etiquetas
                ),
                row=2, col=1
            )
    
    # Actualizar diseño
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80)  # Eliminamos la leyenda del layout
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        <p><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen para df2
def display_summary_table_df2(df, selected_years, selected_plant_types, selected_plants):
    df_clean = prepare_data_df2(df)
    
    selected_years_dt = [pd.to_datetime(str(year)) for year in selected_years]
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt)) & 
        (df_clean['plant_type'].isin(selected_plant_types)) &
        (df_clean['plant_name_ferc1'].isin(selected_plants) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
    ]
    
    # Calcular métricas resumen
    agg_dict = {
        'capex_total': ['sum', 'mean'],
        'opex_production_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }
    
    # Agregar opex_fuel si existe
    if 'opex_fuel' in df_clean.columns:
        agg_dict['opex_fuel'] = ['sum', 'mean']
    
    summary = filtered_df.groupby('plant_type').agg(agg_dict)
    
    # Calcular ratios
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_production_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    # Formatear tabla
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    
    # Seleccionar y renombrar columnas
    columns_to_show = [
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_production_total sum',
        'opex_production_total mean',
        'opex_per_mwh'
    ]
    
    if 'opex_fuel sum' in summary.columns:
        columns_to_show.extend([
            'opex_fuel sum',
            'opex_fuel mean'
        ])
    
    summary = summary[columns_to_show]
    
    # Renombrar columnas
    new_names = {
        'capacity_mw sum': 'Capacidad Total (MW)',
        'net_generation_mwh sum': 'Generación Total (MWh)',
        'capex_total sum': 'CAPEX Total (USD)',
        'capex_total mean': 'CAPEX Promedio (USD)',
        'capex_per_mw': 'CAPEX por MW (USD/MW)',
        'opex_production_total sum': 'OPEX Total (USD)',
        'opex_production_total mean': 'OPEX Promedio (USD)',
        'opex_per_mwh': 'OPEX por MWh (USD/MWh)',
        'opex_fuel sum': 'Costos Combustible Total (USD)',
        'opex_fuel mean': 'Costos Combustible Promedio (USD)'
    }
    
    summary.columns = [new_names.get(col, col) for col in summary.columns]
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos y Métricas</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados.</p>
    </div>
    """
    
    display(HTML(explanation))
    
    # Mostrar tabla con formato
    format_dict = {
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }
    
    if 'Costos Combustible Total (USD)' in summary.columns:
        format_dict.update({
            'Costos Combustible Total (USD)': '${:,.0f}',
            'Costos Combustible Promedio (USD)': '${:,.0f}'
        })
    
    styled_table = summary.style.format(format_dict).background_gradient(
        cmap='Blues', 
        subset=[col for col in summary.columns if 'USD' in col and 'por' not in col]
    )
    
    styled_table = styled_table.set_properties(
        subset=[col for col in summary.columns if 'por' in col or 'por' in col],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos y Métricas"))

## Interfaz de Control para df2
def create_advanced_dashboard_df2(df):
    # Preparar datos
    df_clean = prepare_data_df2(df)
    
    # Obtener opciones para los widgets
    available_years = sorted(df_clean['report_year'].dt.year.unique())
    plant_types = df_clean['plant_type'].unique()
    available_plant_types = sorted([str(pt) for pt in plant_types])
    
    if 'plant_name_with_type' in df_clean.columns:
        plant_names = df_clean['plant_name_with_type'].unique()
        available_plant_names = sorted([str(pn) for pn in plant_names if str(pn) != 'nan'])
    elif 'plant_name_ferc1' in df_clean.columns:
        plant_names = df_clean['plant_name_ferc1'].unique()
        available_plant_names = sorted([str(pn) for pn in plant_names if str(pn) != 'nan'])
    else:
        available_plant_names = ['No disponible']
    
    # Widgets
    year_selector = widgets.SelectMultiple(
        options=['Todos'] + available_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + available_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + available_plant_names,
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    # Función para manejar selección "Todos"
    def get_selected_options(selected, all_options):
        if 'Todos' in selected or len(selected) == 0:
            return all_options
        return selected
    
    # Función para extraer solo el nombre de la planta (sin el tipo entre paréntesis)
    def extract_plant_name(plant_with_type):
        if '(' in plant_with_type:
            return plant_with_type.split('(')[0].strip()
        return plant_with_type
    
    # Función de actualización
    def update_analysis(analysis_type, selected_years, selected_plant_types, selected_plants):
        # Manejar selección "Todos"
        years_to_use = get_selected_options(selected_years, available_years)
        plant_types_to_use = get_selected_options(selected_plant_types, available_plant_types)
        plants_to_use = get_selected_options(selected_plants, available_plant_names)
        
        # Convertir años a enteros
        years_to_use = [int(year) if year != 'Todos' and str(year).isdigit() else year for year in years_to_use]
        
        # Extraer solo el nombre de la planta (sin el tipo entre paréntesis)
        plants_to_use_names = [extract_plant_name(p) for p in plants_to_use]
        
        # Filtrar dataframe
        filtered_df = df_clean[
            (df_clean['report_year'].dt.year.isin(years_to_use)) & 
            (df_clean['plant_type'].isin(plant_types_to_use)) &
            (df_clean['plant_name_ferc1'].isin(plants_to_use_names) if 'plant_name_ferc1' in df_clean.columns else pd.Series([True]*len(df_clean)))
        ]
        
        if analysis_type == 'evolution':
            plot_cost_evolution_df2(filtered_df, years_to_use, plant_types_to_use, plants_to_use_names)
        elif analysis_type == 'composition':
            plot_cost_composition_df2(filtered_df, years_to_use, plant_types_to_use, plants_to_use_names)
        elif analysis_type == 'summary':
            display_summary_table_df2(filtered_df, years_to_use, plant_types_to_use, plants_to_use_names)
    
    # Interfaz interactiva
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector]),
        widgets.HBox([plant_name_selector]),
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type': analysis_type,
            'selected_years': year_selector,
            'selected_plant_types': plant_type_selector,
            'selected_plants': plant_name_selector
        }
    )
    
    display(ui, out)

# Ejecutar el dashboard para df2
create_advanced_dashboard_df2(df2)

Output()

In [62]:
print(df2['plant_type'].unique())

['steam', 'nuclear', NaN, 'combustion_turbine', 'combined_cycle', 'internal_combustion', 'geothermal', 'wind', 'photovoltaic', 'solar_thermal']
Categories (10, object): ['na_category', 'steam', 'wind', 'solar_thermal', ..., 'geothermal', 'photovoltaic', 'combined_cycle', 'internal_combustion']


In [82]:
categorias = sorted(df['plant_name_ferc1'].dropna().unique())
print(categorias)

['(a) mokelumne common', '(a)(f) electra', '(a)(f)salt spring1&2', '(a)(f)tiger creek', '(a)(f)west point', '(b) belden', '(b) butt valley', '(b) caribou no. 1', '(b) caribou no. 2', '(b) cresta', '(b) poe', '(b) rock creek', '(b) stanislaus', '(b)feather river com', '(d) drum no. 1', '(d) drum no.2', '(d) dutch flat', '(d) halsey', '(e) a.g. wishon', '(e) kerckhoff no. 2', '(e)kerckhoff no. 1', '(e-1) balch no.1', '(e-1) haas', '(e-1) kings river', '5', 'a. g. wishon', 'a.g. wishon', 'a.g.wishon', 'afton', 'all facilities', 'all hydro', 'american falls', 'amoskeag', 'balch no. 1', 'balch no. 2', 'bankhead', 'bankhead dam', 'bartletts ferry', 'beardslee', 'belden', 'bellow falls', 'bellows falls', 'bennetts bridge', 'bg crk wtr clct fac', 'big creek 2a', 'big creek 3', 'big creek 4', 'big creek no. 1', 'big creek no. 2', 'big creek no. 2a', 'big creek no. 3', 'big creek no. 4', 'big creek no. 8', 'big creek no.1', 'big creek no.2', 'big creek no.8', 'big creek water collect facilities'

In [87]:
# Asegurarse de que capacity_mw sea numérico
df['capacity_mw'] = pd.to_numeric(df['capacity_mw'], errors='coerce')

# Eliminar filas con valores nulos en capacity_mw o plant_name_ferc1
df = df.dropna(subset=['capacity_mw', 'plant_name_ferc1'])

# Ordenar por capacidad descendente y tomar las 5 primeras
top5 = df.sort_values(by='capacity_mw', ascending=False).head(20)

# Mostrar las columnas clave
print(top5[['report_year', 'plant_name_ferc1',  'capacity_mw']])

      report_year plant_name_ferc1  capacity_mw
258          1994        newcastle  1237.000000
7338         2021         brownlee   675.000000
6953         2023         brownlee   675.000000
7052         2024         brownlee   675.000000
6591         2020         brownlee   675.000000
7353         2022         brownlee   675.000000
6142         2018         brownlee   652.599976
6362         2019         brownlee   652.599976
4137         2010         brownlee   585.400024
4344         2011         brownlee   585.400024
4751         2012         brownlee   585.400024
2207         2001         brownlee   585.400024
4927         2013         brownlee   585.400024
1968         2000         brownlee   585.400024
382          1995         brownlee   585.400024
3833         2008         brownlee   585.400024
3615         2007         brownlee   585.400024
5376         2015         brownlee   585.400024
3132         2005         brownlee   585.400024
2365         2002         brownlee   585

In [ ]:
df2